# DEA Coastlines annual snapshot <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

Generate annual snapshot infographic to accompany annual shoreline release.



### Load packages

First we import all required Python packages, and then start the vector coastline extraction process.

In [2]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os
import sys
import geopandas as gpd
from shapely.geometry import box
from rasterio.transform import array_bounds
import pandas as pd
import shutil
import matplotlib.pyplot as plt

# Load raster generation DEA Coastlines code
sys.path.insert(0, '..')
import dea_coastlines.vector

pd.options.mode.chained_assignment = None

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### All of Australia Year on Year change

In [ ]:
# Summarise by year
prop_df = diff_long.groupby(['year', 'class']).count()
prop_df = prop_df.groupby(['year']).transform(lambda x: (x / x.sum()) * 100)

# Pivot to make classes columns
pivot_df = prop_df.drop('region', axis=1).rename({'distance': 'proportion'}, axis=1).reset_index('class').pivot(columns='class', values='proportion')

In [ ]:
import matplotlib.colors as colors

def get_continuous_cmap(hex_list, float_list=None):
    ''' creates and returns a color map that can be used in heat map figures.
        If float_list is not provided, colour map graduates linearly between each color in hex_list.
        If float_list is provided, each color in hex_list is mapped to the respective location in float_list. 

        Parameters
        ----------
        hex_list: list of hex code strings
        float_list: list of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.

        Returns
        ----------
        colour map'''


    def hex_to_rgb(value):
        '''
        Converts hex to rgb colours
        value: string of 6 characters representing a hex colour.
        Returns: list length 3 of RGB values'''
        value = value.strip("#") # removes hash symbol if present
        lv = len(value)
        return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

    def rgb_to_dec(value):
        '''
        Converts rgb to decimal colours (i.e. divides each value by 256)
        value: list (length 3) of RGB values
        Returns: list (length 3) of decimal values'''
        return [v/256 for v in value]

    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))

    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = colors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
    return cmp

    
fig = plt.figure(figsize=(13, 7.5))
gs = fig.add_gridspec(2, 3, 
                      width_ratios=[0.4, 1.3, 1.0],
                      height_ratios=[0.8, 1], wspace=0.0, hspace=0.0)
ax_time = fig.add_subplot(gs[:, 0])
ax_text = fig.add_subplot(gs[0, 1:3])
ax_pie = fig.add_subplot(gs[1, 1])
ax_map = fig.add_subplot(gs[1, 2])

for year in range(1990, 2021):
# for year in range(2020, 2021):
    
    ################
    # Text summary #
    ################
    
    # Hide axis to create blank box
    ax_text.axis('off')
    
    # Add plot title and year
    ax_text.annotate('Coastal change in Australia'.upper(), 
                     xy=(0.03, 0.99), 
                     xycoords='axes fraction', 
                     va='top', 
                     fontsize=20, 
                     fontweight='bold')
    ax_text.annotate(year, 
                     xy=(0.97, 1.0), 
                     xycoords='axes fraction', 
                     ha='right', 
                     va='top', 
                     fontsize=65, 
                     fontweight='bold')
    
    # Calculate year-on-year change and set up symbols
    symbols = [u' \N{BLACK DOWN-POINTING TRIANGLE} down ', u' \N{BLACK UP-POINTING TRIANGLE} up ']
    absolute = pivot_df.loc[year]
    diffs = (pivot_df.loc[year] - pivot_df.loc[year - 1])
    triangles = pd.cut(diffs, [-100, 0, 100], labels=symbols)

    # Add retreat summary text
    retreat_text = (f"of Australia's non-rocky coastlines\nretreated by > 10 metres this year\n({triangles.iloc[0]}{diffs.abs().iloc[0]:.1f}% from {year - 1})")
    ax_text.annotate(f'{absolute.iloc[0]:.1f}%', (0.18, 0.76), xycoords='axes fraction', ha='right', va='top', fontsize=28, fontweight='bold')
    ax_text.annotate(retreat_text, (0.20, 0.77), xycoords='axes fraction', ha='left', va='top', fontsize=13)
    
    # Add grow summary text
    grow_text = (f"grew by > 10 metres this year\n({triangles.iloc[-1]}{diffs.abs().iloc[-1]:.1f}% from {year - 1})")
    ax_text.annotate(f'{absolute.iloc[-1]:.1f}%', (0.18, 0.41), xycoords='axes fraction', ha='right', va='top', fontsize=28, fontweight='bold')
    ax_text.annotate(grow_text, (0.20, 0.42), xycoords='axes fraction', ha='left', va='top', fontsize=13)
    
    # Add DEA logo overlay
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    from matplotlib.image import imread
    im_arr = imread('/g/data/r78/DEACoastlines/visualisation/images/dea_logo.png')
    imagebox = OffsetImage(im_arr, zoom=0.18)
    ab = AnnotationBbox(imagebox, 
                        xy=(0.68, 0.58), 
                        xycoords='axes fraction', 
                        box_alignment=(0, 1), 
                        frameon=False) 
    ax_text.add_artist(ab) 
    
    
    ##############
    # Timeseries #
    ##############

    # Cumulative plot of each year-on-year change
    pivot_df.loc[1989:year].plot.area(cmap='RdBu', 
                                      linewidth=0.6,
                                      ax=ax_time, legend=False)  
    for i, li in enumerate(ax_time.lines):
        li.set_color('black')        
        if i < 3:
            li.set_ydata(li.get_ydata() - 0.45)
    ax_time.axhline(50, c='black', linestyle='dashed')
    ax_time.set_xlim([1989, year])
    ax_time.set_ylim([0, 100])
    import matplotlib.ticker as mtick
    ax_time.yaxis.set_major_formatter(mtick.PercentFormatter())
    ax_time.ticklabel_format(useOffset=False, axis='x', style='plain')
    ax_time.xaxis.set_major_formatter(mtick.FormatStrFormatter('%g'))
    ax_time.set_xlabel(None)
    
    # Add retreat and growth annotations
    ax_time.annotate('Growth', (0.50, 0.965), xycoords='axes fraction', ha='center', va='center', color='white', fontsize=12, fontweight='bold')
    ax_time.annotate('Retreat', (0.50, 0.025), xycoords='axes fraction', ha='center', va='center', color='white', fontsize=12, fontweight='bold')
    
    
    ############
    # Pie plot #
    ############

    # Plot single year of data as a pie plot
    year_df = pivot_df.loc[year]
    year_df.plot.pie(cmap='RdBu', wedgeprops={"edgecolor":"black",'linewidth': 1.0, 'antialiased': True, "width": 0.5}, 
                     startangle=90, 
                     counterclock=True, 
                     fontsize=12,
                     autopct=lambda p:f'{p:.0f}%',
                     pctdistance=0.75, 
                     textprops={'fontsize': 14},
                     ax=ax_pie)
    
    # Add title and annotations
    ax_pie.annotate(year, (0.5, 0.5), xycoords='axes fraction', ha='center', va='center', fontsize=15)
    ax_pie.annotate(("Results based on year-on-year change in the median annual position of the\n"
                     "shoreline mapped by Digital Earth Australia Coastlines; Bishop-Taylor et al.\n(2021). "
                     "Available at https://doi.org/10.1016/j.rse.2021.112734 (CC BY 4.0)"), 
                    (-0.10, 0.04), xycoords='axes fraction', ha='left', va='top', fontsize=7.8, color='grey')
    ax_pie.set_title(" ".join(f"Shoreline change in {year}".upper()), fontsize=11, pad=10)
    ax_pie.set_anchor('N')
    ax_pie.set_ylabel(None);   
    ax_pie.texts[1].set_color('white')
    ax_pie.texts[-3].set_color('white')
    
    
    ############
    # Plot map #
    ############    

    # Select average rate of change for year and join to coastal regions data
    region_df = region_rate_df.xs(year, level='year').round(1)
    acsc_regions_results = acsc_regions.join(region_df)
    
    # Create a custom inset for legend
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    cax = inset_axes(ax_map,
                 width=1.1,  
                 height=0.2,
                 bbox_to_anchor=[0.72, 0.12],
                 bbox_transform=plt.gcf().transFigure,
                 loc='center'
                   )
    
    # Plot coastal regions data     
    cmap = get_continuous_cmap(hex_list=['#67001f','#b2182b','#d6604d','#f4a582','#fddbc7',
                                         '#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061'], 
                               float_list=[0, 0.10, 0.30, 0.40, 0.45, 0.5, 0.55, 0.60, 0.70, 0.90, 1.0])

    acsc_regions_results.plot(column='distance', 
                              cmap=cmap,
                              vmin=-5, 
                              vmax=5, 
                              ax=ax_map, 
                              edgecolor='black', 
                              linewidth=1,
                              legend=True,
                              cax=cax,
                              legend_kwds={'label': None, 
                                           'orientation': "horizontal",
                                           'ticks': []})
    cax.set_title('Median shoreline change', pad=9, y=-20)
    cax.get_yaxis().set_ticks([])
    for loc, lab in zip([-7.1, 0, 7.1], ['-5 m', '0 m', '+5 m']):
        cax.text(loc, -0.8, lab, ha='center', va='center', color='black')
    
    ax_map.annotate(f"{acsc_regions_results.loc['South-eastern Coasts'].distance:.1f} m", 
                    xy=(150.5, -36), 
                    xytext=(154, -42), 
                    arrowprops=dict(arrowstyle='-', lw=0.6),
                    fontsize=12,
                    va='center', ha='center')
    
    ax_map.annotate(f"{acsc_regions_results.loc['North-eastern Coasts'].distance:.1f} m", 
                xy=(145.49898, -17.10035), 
                xytext=(153, -14), 
                arrowprops=dict(arrowstyle='-', lw=0.6),
                fontsize=12,
                va='center', ha='center')
    
    ax_map.annotate(f"{acsc_regions_results.loc['Carpentaria Gulf Coasts'].distance:.1f} m        ", 
            xy=(139, -17), 
            xytext=(139, -8.5), 
            arrowprops=dict(arrowstyle='-', lw=0.6),
            fontsize=12,
            va='center', ha='center')
    
    ax_map.annotate(f"{acsc_regions_results.loc['North-western Coasts'].distance:.1f} m", 
            xy=(122.82193, -16.79609), 
            xytext=(119.5, -13), 
            arrowprops=dict(arrowstyle='-', lw=0.6),
            fontsize=12,
            va='center', ha='center')
    
    ax_map.annotate(f"{acsc_regions_results.loc['Western Coasts'].distance:.1f} m", 
            xy=(115.2, -30), 
            xytext=(111.8, -30), 
            arrowprops=dict(arrowstyle='-', lw=0.6),
            fontsize=12,
            va='center', ha='right')
    
    ax_map.annotate(f"{acsc_regions_results.loc['Southern Coasts'].distance:.1f} m", 
            xy=(128.5, -32), 
            xytext=(128.5, -37), 
            arrowprops=dict(arrowstyle='-', lw=0.6),
            fontsize=12,
            va='center', ha='center')
    
    # Add annotions and title
    ax_map.annotate('Australia', (0.48, 0.55), xycoords='axes fraction', ha='center', va='center', fontsize=15)
    ax_map.set_title(" ".join("Change by coastal region".upper()), fontsize=11, pad=10)
    ax_map.set_anchor('N')
    ax_map.axis('off');
    
    
    ################
    # Export image #
    ################
    
    fig.savefig(f'deacoastlines_annualupdate_{year}.png', bbox_inches='tight', pad_inches=0.1, dpi=111.5)
    
    ax_text.cla()
    ax_map.cla()
    ax_pie.cla()
    ax_time.cla()
    fig.delaxes(cax)


In [ ]:
!ffmpeg -y -r 6 -start_number 1990 -i deacoastlines_annualupdate_%4d.png -c:v libx264 -vf "fps=6,format=yuv420p,pad=ceil(iw/2)*2:ceil(ih/2)*2" temp.mp4
!ffmpeg -y -i temp.mp4 -filter_complex "[0]trim=0:5[hold],[0][hold]concat[extended],[extended][0]overlay" -pix_fmt yuv420p -c:a copy temp2.mp4
!ffmpeg -y -i temp2.mp4 -filter_complex "[0]trim=0:5[hold],[0][hold]concat[extended],[extended][0]overlay" -pix_fmt yuv420p -c:a copy deacoastlines_annualupdate_{year}.mp4

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/DEACoastLines/issues/new). For questions or more information about this product, sign up to the [Open Data Cube Slack](https://join.slack.com/t/opendatacube/shared_invite/zt-d6hu7l35-CGDhSxiSmTwacKNuXWFUkg) and post on the [`#dea-coastlines`](https://app.slack.com/client/T0L4V0TFT/C018X6J9HLY/details/) channel.

**Last modified:** July 2021